In [284]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC #selenium에서 사용할 모듈 import
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException

In [319]:
import time
import requests
from bs4 import BeautifulSoup

import re
import csv
import pandas as pd

In [326]:
driver = webdriver.Chrome("C:\Temp\chromedriver\chromedriver.exe")
url = "https://map.naver.com/v5/search"
driver.get(url)
try:
    element = WebDriverWait(driver,10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
    ) #입력창이 뜰 때까지 대기
finally:
    pass
    
    
#try:
    #f = open('foo.txt', 'w')
    # 무언가를 수행한다.

    #(... 생략 ...)
#finally:
    #f.close()  # 중간에 오류가 발생하더라도 무조건 실행된다.

In [327]:
search_box = driver.find_element(By.CLASS_NAME,"input_search")
search_box.send_keys("전남대 상대 맛집")
search_box.send_keys(Keys.ENTER) #검색창에 "전남대 맛집" 입력

In [330]:
time.sleep(7) #화면 표시 기다리기
frame = driver.find_element_by_css_selector("iframe#searchIframe")
driver.switch_to.frame(frame)

time.sleep(3)
# 여기까지 iframe 전환

In [331]:
scroll_div = driver.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]')
#검색 결과로 나타나는 scroll-bar 포함한 div 잡고
driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
#여기까지 scroll
#맨 아래까지 내려서 해당 페이지의 내용이 다 표시되게 함
#//*[@id="_pcmap_list_scroll_container"]

In [332]:
#csv 파일 생성
file = open('12.csv', mode='w', newline='')
writer = csv.writer(file)
writer.writerow(["place", "rate","address","image","menu1","menu1img"])
final_result = []
time.sleep(1)

In [343]:
i = 2
while i<=5: 
    stores_box = driver.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]/ul')
    stores = driver.find_elements_by_css_selector("li")
    #_pcmap_list_scroll_container > ul > li:nth-child(1)
    #해당 페이지에서 표시된 모든 가게 정보
    for store in stores: #한 페이지 내에서의 반복문. 순차적으로 가게 정보에 접근
        name = store.find_element_by_css_selector("div.CHC5F > div.eY7cJ > a > div > div > span.place_bluelink.TYaxT").text #가게 이름
        #_pcmap_list_scroll_container > ul > li:nth-child(1) > div.CHC5F > div.eY7cJ > a > div > div > span.place_bluelink.TYaxT
        #app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot.LXIwF > em
        #리뷰 정보
        try:    
            rating = re.search(r'/span>(\d+\.?\d*)', store.find_element_by_css_selector("div.CHC5F > div.Dr_06 > div > span.h69bs.a2RFq").get_attribute('innerHTML')).groups()[0]
            #_pcmap_list_scroll_container > ul > li:nth-child(1) > div.CHC5F > div.Dr_06 > div > span.h69bs.a2RFq
            #_pcmap_list_scroll_container > ul > li:nth-child(1) > div.CHC5F > div.Dr_06 > div > span.h69bs.a2RFq > span
        except:
            rating = ''
        time.sleep(3)
            # 평점 숫자 부분만 rating에 담음. 평점이 없는 경우가 있어 예외 처리
        #이미지 정보 확인
        try:
            #img_src = re.findall(r'url\("([^"]+)"\)', store.find_element_by_css_selector("div.yLaWz.FlUUg.bnOAZ > div > div > div > a:nth-child(1) >div > div").get_attribute('style')).groups()[0]
            #_pcmap_list_scroll_container > ul > li:nth-child(1) > div.yLaWz.FlUUg.bnOAZ > div > div > div > a:nth-child(1) > div > div
            #_pcmap_list_scroll_container > ul > li:nth-child(3) > div.yLaWz.FlUUg.bnOAZ > div > div > div > a:nth-child(1) > div > div
            style_attribute = store.find_element_by_css_selector("div.yLaWz.FlUUg.bnOAZ > div > div > div > a:nth-child(1) >div > div").get_attribute('style')
            img_src = re.findall(r'url\("([^"]+)"\)', style_attribute)[0]  # 첫 번째 사진 URL 가져오기
        except:
            img_src = ''
            #역시 대표 이미지가 없는 경우가 있어 예외 처리
        click_name = store.find_element_by_css_selector("div.CHC5F > div.eY7cJ > a > div > div > span.place_bluelink.TYaxT")
        click_name.click() 
        # 가게 주소, 홈페이지 링크를 확인하려면 가게 이름을 클릭해 세부 정보를 띄워야 함.
        driver.switch_to.default_content()
        time.sleep(7)
        frame_in = driver.find_element_by_xpath('//*[@id="entryIframe"]')
        #//*[@id="entryIframe"]
        driver.switch_to.frame(frame_in) 
        time.sleep(3)
        #주소 정보 확인
        try:
            #address = re.search('광주\s(.+)',driver.find_element_by_css_selector("#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin.vKA6F > div > div > div.O8qbU.tQY7D > div > a > span.LDgIH")).text.groups()[0]
            #app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin.vKA6F > div > div > div.O8qbU.tQY7D > div > a > span.LDgIH
            address_element = driver.find_element_by_css_selector("#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin.vKA6F > div > div > div.O8qbU.tQY7D > div > a > span.LDgIH")
            address_text = address_element.text
            address = re.findall(r'광주\s(.+)', address_text)
        except:
            address = ''
        #메뉴 추출
        try:
            driver.execute_script("window.scrollTo(0,2000);")
            time.sleep(2)
        except:
            menu1 = ''
            #menu1img = ' '
        #메뉴 1 이름 확인
#         try:
#             munu1 = store.find_element_by_css_selector("a > div.MN48z > div.erVoL > div").text
#             #app-root > div > div > div > div:nth-child(7) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(1) > a > div.MN48z > div.erVoL > div
#             #a > div.ZHqBk > div > img
#             #app-root > div > div > div > div:nth-child(7) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(1) > a > div.ZHqBk > div > img
#         except:
#             munu1 = ''
        store_info = {
            'placetitle': name,
            'rate': rating,
            'address': address,
            'image': img_src,
            'menu1' : menu1
        }


    #크롤링한 정보들을 store_info에 담고
        print(name,rating,address,img_src,menu1)
        print("*" * 50)
        final_result.append(store_info)
                    # 출력해서 확인 후 final_result에 저장

        driver.switch_to.default_content()
        driver.switch_to.frame(frame)
        time.sleep(8)

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_14000/448543333.py, line 79)

In [ ]:
for result in final_result: #크롤링한 가게 정보에 순차적으로 접근 & csv 파일 작성
    row = []
    row.append(result['placetitle'])
    row.append(result['rate'])
    row.append(result['address'])
    row.append(result['image'])
    writer.writerow(row)
file.close()

In [212]:
print(final_result)

[{'placetitle': '피자스쿨 광주전대상대점', 'rate': '4.45', 'address': ['북구 반룡로 25 1층'], 'image': ['https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200405_99%2F1586015464813Aoipz_JPEG%2F2fjK8subE_QGIzIFz2um3jLh.jpeg.jpg']}, {'placetitle': '오늘하루가 전대상대점', 'rate': '4.94', 'address': ['북구 반룡로27번길 17'], 'image': ['https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221207_72%2F1670411893274cWpg3_JPEG%2F%25B0%25A3%25C6%25C7%25BB%25E7%25C1%25F81.jpeg']}, {'placetitle': '마니마니전대상대점', 'rate': '4.08', 'address': ['북구 설죽로202번길 100'], 'image': ''}, {'placetitle': '국수나무 광주전대상대점', 'rate': '4.57', 'address': ['북구 반룡로27번길 7'], 'image': ''}, {'placetitle': '메가MGC커피 전대상대점', 'rate': '4.42', 'address': ['북구 설죽로202번길 64 1층'], 'image': ''}, {'placetitle': '얌얌 전대상대점', 'rate': '4.63', 'address': ['북구 설죽로214번길 115'], 'image': ''}, {'placetitle': '진심커피 전남대상대점', 'rate': '', 'address': ['북구 설죽로202번길 68